<a href="https://colab.research.google.com/github/praveen-soni06/Image-Based-Breed-Recognition-System-SIH-25004-/blob/main/sih.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [4]:
# DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/My Drive/cow_datasets"
print(os.listdir(data_path))

In [ ]:
images = []
labels = []

for breed_folder in os.listdir(data_path):
    breed_path = os.path.join(data_path, breed_folder)
    if os.path.isdir(breed_path):
        for image_name in os.listdir(breed_path):
            img_path = os.path.join(breed_path, image_name)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is not None:
                img = cv2.resize(img, (224, 224))  # Bigger size for ResNet
                images.append(img)
                labels.append(breed_folder)

X = np.array(images, dtype="float32") / 255.0
y = np.array(labels)


In [ ]:
# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)
num_classes = len(np.unique(y))

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# TRANSFORMS
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

In [ ]:
# CUSTOM DATASET
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        img = self.features[idx].astype(np.uint8)
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return img, label


In [ ]:
# Datasets
train_dataset = CustomDataset(X_train, y_train, transform=train_transform)
test_dataset = CustomDataset(X_test, y_test, transform=test_transform)

In [ ]:
# MODEL
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace final layer
model = model.to(device)


In [ ]:
# LOSS, OPTIMIZER, SCHEDULER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

In [ ]:
# TRAINING LOOP
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

In [ ]:
model.eval()

In [ ]:
# EVALUATION FUNCTION
def evaluate(model, loader, device):
    correct_top1, correct_top3, total = 0, 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = torch.max(outputs, 1)
            _, top3_preds = outputs.topk(3, 1)
            correct_top1 += (preds == labels).sum().item()
            for i in range(labels.size(0)):
                if labels[i] in top3_preds[i]:
                    correct_top3 += 1
            total += labels.size(0)
    return 100*correct_top1/total, 100*correct_top3/total

In [ ]:
# EVALUATE
test_top1, test_top3 = evaluate(model, test_loader, device)
train_top1, train_top3 = evaluate(model, train_loader, device)

print(f"Test Set - Top-1 Accuracy: {test_top1:.2f}%, Top-3 Accuracy: {test_top3:.2f}%")
print(f"Train Set - Top-1 Accuracy: {train_top1:.2f}%, Top-3 Accuracy: {train_top3:.2f}%")

In [ ]:
# SINGLE IMAGE PREDICTION
def predict_breed(model, img_path, device, label_encoder):
    model.eval()
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
    breed_name = label_encoder.inverse_transform(pred.cpu().numpy())[0]
    return breed_name


predicted_breed = predict_breed(model, "/content/drive/My Drive/cow_datasets/Breed1/img1.jpg", device, le)
print("Predicted Breed:", predicted_breed)